In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import itertools

In [2]:
with open('numpy arrays/X.npy', 'rb') as f:
    X = np.load(f, allow_pickle=True)
    print('X loaded')
with open('numpy arrays/y.npy', 'rb') as f:
    y = np.load(f, allow_pickle=True)
    print('y loaded')

X loaded
y loaded


In [3]:
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

In [4]:
X = np.stack((X,)*3, axis=-1)
print(X.shape)

(17615, 100, 100, 3)


In [5]:
new_X = tf.image.resize(
    X, [100,100], method=tf.image.ResizeMethod.BILINEAR, preserve_aspect_ratio=False,
    antialias=False, name=None
)
new_X.shape
X = np.array(new_X)

In [6]:
from tensorflow.keras.applications import VGG16,VGG19,NASNetMobile,DenseNet201
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import Adam 

In [7]:
# Using early stopping and reduce lr on plateau to control the performance
cb_1 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-5, verbose=0) 
cb_2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=10, verbose=1, mode='auto')

cbs = [cb_1,cb_2]

In [8]:
from sklearn.model_selection import train_test_split
# split the data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state = 42, stratify=y) 

In [9]:
model = Sequential()
# tf.keras.applications.InceptionResNetV2(
#     include_top=True, weights='imagenet', input_tensor=None,
#     input_shape=None, pooling=None, classes=1000,
#     classifier_activation='softmax', **kwargs
# )
conv_base = tf.keras.applications.InceptionResNetV2(input_shape=(100,100,3), include_top=False, pooling='max',weights='imagenet')
conv_base.trainable = False
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax')) 

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
  if layer in train_layers:
    layer.trainable = True

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])

In [10]:
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1536)              54336736  
_________________________________________________________________
batch_normalization_203 (Bat (None, 1536)              6144      
_________________________________________________________________
dense (Dense)                (None, 2048)              3147776   
_________________________________________________________________
batch_normalization_204 (Bat (None, 2048)              8192      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 10245     
Total params: 57,509,093
Trainable params: 3,165,189
Non-trainable params: 54,343,904
_________________________________________________________________


In [11]:
history = model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=8, callbacks=cbs,verbose=1) 

Epoch 1/8
413/413 [==============================] - 237s 547ms/step - loss: 437.1188 - accuracy: 0.4404 - val_loss: 37.8968 - val_accuracy: 0.4376
Epoch 2/8
413/413 [==============================] - 176s 426ms/step - loss: 20.7407 - accuracy: 0.4098 - val_loss: 7.4120 - val_accuracy: 0.4037
Epoch 3/8
413/413 [==============================] - 173s 419ms/step - loss: 6.7017 - accuracy: 0.3776 - val_loss: 4.9947 - val_accuracy: 0.4114
Epoch 4/8
 84/413 [=====>........................] - ETA: 1:44 - loss: 5.5642 - accuracy: 0.3767